# Spread of information in a social network

An illustrative social simulation involves five users on a social media platform, simulating how information spreads on the day of a major event. Two users, Alice and Bob, are influential figures, each sharing content to promote their views on the event. A third user, Charlie, is intentionally spreading misinformation to harm Alice's credibility. The last two users, Dana and Evan, represent ordinary users who don’t have any specific agenda. They interact with the platform by liking, sharing, or commenting on content, but their behavior is influenced by what they see from Alice, Bob, and Charlie. Throughout the simulation, the spread of information, user influence, and engagement levels are tracked to understand how social influence shapes the narrative.

## Setup and imports

In [3]:
# Libraries
import collections
import concurrent.futures
import datetime
import random
import matplotlib.pyplot as plt
import termcolor
import sentence_transformers
from collections.abc import Callable, Sequence
from IPython import display

# Concordia modules
from concordia.agents import entity_agent
from concordia import components as generic_components
from concordia.metrics import goal_achievement
from concordia.agents import deprecated_agent as basic_agent
from concordia.metrics import common_sense_morality
from concordia.components.agent import to_be_deprecated as components
from concordia.components import game_master as gm_components
from concordia.document import interactive_document
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.metrics import opinion_of_others
from concordia.clocks import game_clock
from concordia.environment import game_master
from concordia.language_model import gpt_model
from concordia.language_model import language_model
from concordia.metrics.v2 import context_free_rationality
from concordia.metrics.v2 import context_free_common_sense_morality
from concordia.typing import agent as agent_lib
from concordia.typing import component
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting
from concordia.agents import entity_agent_with_logging
from concordia.components import agent as agent_components
from concordia.memory_bank import legacy_associative_memory

In [4]:
# Setup sentence encoder
st_model = sentence_transformers.SentenceTransformer(
    'sentence-transformers/all-mpnet-base-v2')
embedder = lambda x: st_model.encode(x, show_progress_bar=False)

c:\Users\algar\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Setup LLM
GPT_API_KEY = 'lm-studio' #@param {type: 'string'}
GPT_MODEL_NAME = 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF' #@param {type: 'string'}

if not GPT_API_KEY:
  raise ValueError('GPT_API_KEY is required.')

model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY,
                                   model_name=GPT_MODEL_NAME)

## Configuring the generic knowledge of players and GM

This generic knowledge will be the memories that all players and GM share (common context).

In [6]:
shared_memories = [
    'There is a social media platform called ConnectNet.',
    'ConnectNet is a popular platform for sharing news and personal updates.',
    'Users on ConnectNet often follow trends based on viral content.',
    'The platform has features like posts, comments, likes, and shares.',
    'Trending topics on ConnectNet can quickly gain attention and spread across the user base.',
    'The platform allows users to follow others and interact with their posts.',
    'There are no official fact-checkers on ConnectNet, so misinformation can spread quickly.',
    'The platform has a user-friendly interface, encouraging high levels of engagement.',
    'Popular posts on ConnectNet often include viral challenges, news updates, and opinion pieces.',
    'Algorithms on ConnectNet prioritize content based on user engagement, amplifying popular posts.',
]

shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)

print(shared_context)
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

ConnectNet is a social media platform where users share news and personal updates, engage with trends, and interact through features like posts, comments, likes, and shares. Its user-friendly interface encourages high levels of engagement, leading to the quick spread of viral content, including misinformation, as there are no official fact-checkers on the platform. Popular posts include challenges, news updates, and opinion pieces, with algorithms prioritizing content based on user engagement.


In [7]:
# Make the clock
SETUP_TIME = datetime.datetime(hour=8, year=2024, month=9, day=1)

START_TIME = datetime.datetime(hour=9, year=2024, month=10, day=1)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[datetime.timedelta(hours=1), datetime.timedelta(seconds=10)])

DEFAULT_PLANNING_HORIZON = 'the rest of the day, focusing most on the near term'

## Functions to build the players

Normally, agents are made up of these elements:

1. Associative memory: saves all observations, and does retrieval of semantically relevant memories on request.
2. Formative memory: allows customization of traits, specific memories, and even details like the agent's birthday.
3. Component: represents a part of the agent's thought process. Retrieves items from associative memory and formats or summarizes them in a way that makes them useful as context, either for another component or for action selection.

In [8]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [9]:
DEFAULT_PLANNING_HORIZON = 'the rest of the day, focusing most on the near term'

def _get_class_name(object_: object) -> str:
  return object_.__class__.__name__

In [10]:
def build_agent(agent_config,
                    player_names: list[str],
                    measurements: measurements_lib.Measurements | None = None):

  mem = formative_memory_factory.make_memories(agent_config)
  raw_memory = legacy_associative_memory.AssociativeMemoryBank(mem)

  agent_name = agent_config.name

  instructions = agent_components.instructions.Instructions(
      agent_name=agent_name,
      logging_channel=measurements.get_channel('Instructions').on_next,
  )

  time_display_label = '\nCurrent time'
  time_display = agent_components.report_function.ReportFunction(
      function=clock.current_time_interval_str,
      pre_act_key=time_display_label,
      logging_channel=measurements.get_channel('TimeDisplay').on_next,
  )
  somatic_state_label = '\nSensations and feelings'
  somatic_state = (
          agent_components.question_of_query_associated_memories.SomaticState(
              model=model,
              clock_now=clock.now,
              logging_channel=measurements.get_channel('SomaticState').on_next,
              pre_act_key=somatic_state_label,
  ))
  identity_label = '\nIdentity characteristics'
  identity = agent_components.question_of_query_associated_memories.Identity(
      model=model,
      logging_channel=measurements.get_channel('Identity').on_next,
      pre_act_key=identity_label,
  )
  observation_label = '\nObservation'
  observation = agent_components.observation.Observation(
      clock_now=clock.now,
      timeframe=clock.get_step_size(),
      pre_act_key=observation_label,
      logging_channel=measurements.get_channel('Observation').on_next,
  )
  goal_key = 'Goal'
  goal_label = '\nOverarching goal'
  overarching_goal = agent_components.constant.Constant(
      state=agent_config.goal,
      pre_act_key=goal_label,
      logging_channel=measurements.get_channel(goal_label).on_next)
  plan = agent_components.plan.Plan(
      model=model,
      observation_component_name=_get_class_name(observation),
      components={_get_class_name(identity): identity_label,},
      clock_now=clock.now,
      goal_component_name=goal_key,
      horizon=DEFAULT_PLANNING_HORIZON,
      pre_act_key='\nPlan',
      logging_channel=measurements.get_channel('Plan').on_next,
  )

  observation_summary_label = '\nSummary of recent observations'
  observation_summary = agent_components.observation.ObservationSummary(
      model=model,
      clock_now=clock.now,
      timeframe_delta_from=datetime.timedelta(hours=4),
      timeframe_delta_until=datetime.timedelta(hours=0),
      pre_act_key=observation_summary_label,
      logging_channel=measurements.get_channel('ObservationSummary').on_next,
  )
  rationality_metric = (
      context_free_rationality.RationalityMetric(
          model=model,
          player_goal=agent_config.goal,
          clock=clock,
          logging_channel=measurements.get_channel(
              'RationalityMetric').on_next,
          measurements=measurements,
  ))
  morality_metric = (
      context_free_common_sense_morality.CommonSenseMoralityMetric(
          model=model,
          clock=clock,
          logging_channel=measurements.get_channel(
              'CommonSenseMorality').on_next,
          measurements=measurements,
  ))
  entity_components = (
      # Components that provide pre_act context.
      instructions,
      identity,
      plan,
      somatic_state,
      observation_summary,
      observation,
      time_display,
      rationality_metric,
      morality_metric,
  )
  components_of_agent = {_get_class_name(component): component
                         for component in entity_components}
  components_of_agent[goal_key] = overarching_goal
  components_of_agent[
      agent_components.memory_component.DEFAULT_MEMORY_COMPONENT_NAME] = (
          agent_components.memory_component.MemoryComponent(raw_memory))

  act_component = agent_components.concat_act_component.ConcatActComponent(
      model=model,
      clock=clock,
      logging_channel=measurements.get_channel('ActComponent').on_next,
  )
  agent = entity_agent_with_logging.EntityAgentWithLogging(
      agent_name=agent_name,
      act_component=act_component,
      context_components=components_of_agent,
      component_logging=measurements,
  )

  return agent, mem

## Configure and build the players

In [11]:
def make_random_big_five()->str:
  return str({
      'extraversion': random.randint(1, 10),
      'neuroticism': random.randint(1, 10),
      'openness': random.randint(1, 10),
      'conscientiousness': random.randint(1, 10),
      'agreeableness': random.randint(1, 10),
  })

player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Grow your follower base by 20% and boost engagement through interactive posts.',
        context=shared_context+'You are an influencer who wants to build a loyal following by sharing engaging content and fostering community interaction on the platform.',
        traits = make_random_big_five(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
    ),
    formative_memories.AgentConfig(
        name='Bob',
        gender='male',
        goal='Create at least three viral posts and collaborate with other influencers to increase reach.',
        context=shared_context+'You are an influencer who wants to enhance your influence by creating captivating content that resonates with a diverse audience.',
        traits=make_random_big_five(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
    ),
    formative_memories.AgentConfig(
        name='Charlie',
        gender='male',
        goal='Discredit Alice by creating compelling but false narratives that gain traction among users.',
        context=shared_context+"You spread misinformation to damage Alice's credibility and sow confusion.",
        traits=make_random_big_five(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
    ),
    formative_memories.AgentConfig(
        name='Dana', 
        gender='female',
        goal='Engage with interesting content daily and share posts that resonate with your friends.',
        context=shared_context+'You are a regular user who engages with content on the platform by liking, sharing, or commenting based on what you see.',
        traits=make_random_big_five(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
    ),
    formative_memories.AgentConfig(
        name='Evan', 
        gender='male',
        goal='Become a go-to source for trending news among your peers by sharing reliable information.',
        context=shared_context+'You are a regular user who engages with content on the platform by liking, sharing, or commenting based on what you see.',
        traits=make_random_big_five(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
    ),
]

In [12]:
NUM_PLAYERS = len(player_configs)

player_configs = player_configs[:NUM_PLAYERS]
player_goals = {
    player_config.name: player_config.goal for player_config in player_configs}
players = []
memories = {}
measurements = measurements_lib.Measurements()

player_names = [player.name for player in player_configs][:NUM_PLAYERS]
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent, mem in pool.map(build_agent,
                             player_configs[:NUM_PLAYERS],
                             # All players get the same `player_names`.
                             [player_names] * NUM_PLAYERS,
                             # All players get the same `measurements` object.
                             [measurements] * NUM_PLAYERS):
    players.append(agent)
    memories[agent.name] = mem

c:\Users\algar\Documents\Travail\GitHub\ConcordiaSims\concordia\associative_memory\associative_memory.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(
c:\Users\algar\Documents\Travail\GitHub\ConcordiaSims\concordia\associative_memory\associative_memory.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(
c:\Users\algar\Documents\Travail\GitHub\ConcordiaSims\concordia\associative_memory\associative_memory.py:128: FutureWarning: T

## Build GM

In [12]:
# @title Define the election component

class Elections(component.Component):
  """Tracks elections."""

  def __init__(
      self,
      model: language_model.LanguageModel,
      memory: associative_memory.AssociativeMemory,
      voters: Sequence[entity_agent.EntityAgent],
      candidates: Sequence[str],
      clock_now: Callable[[], datetime.datetime],
      verbose: bool = False,
      measurements: measurements_lib.Measurements | None = None,
      channel: str = 'election',
  ):
    """Initializes the election tracker.

    Args:
      model: The language model to use.
      memory: The memory to use.
      voters: The agent voters.
      candidates: The candidates in the election.
      clock_now: Function to call to get current time. Used for logging.
      verbose: Whether to print verbose messages.
      measurements: Optional object to publish data from the elections.
      channel: Channel in measurements to publish to.
    """
    self._model = model
    self._memory = memory
    self._voters = voters
    self._candidates = candidates
    self._clock_now = clock_now
    self._verbose = verbose
    self._measurements = measurements
    self._channel = channel

    self._voter_names = [voter.name for voter in self._voters]
    self._vote_count = {candidate: 0 for candidate in self._candidates}
    self._citizens_who_already_voted = set()

    self._voter_by_name = {voter.name: voter for voter in self._voters}
    self._state = 'Polls are not open yet.'
    self._partial_states = None
    self._history = []

    self._polls_open = False
    self._winner_declared = False
    self._timestep = 0

  def get_last_log(self):
    if self._history:
      return self._history[-1].copy()

  def get_history(self):
    return self._history.copy()

  def open_polls(self) -> None:
    self._polls_open = True
    self._state = 'Polls are open, voting in progress.'

  def declare_winner(self) -> None:
    if not self._winner_declared:
      self._winner_declared = True
      self._polls_open = False
      winner = max(self._vote_count, key=self._vote_count.get)
      self._state = f'Polls are closed. {winner} won the election.'
      if self._verbose:
        print(termcolor.colored('\n' + self._state, 'red'), end='')

      self._memory.add(self._state, tags=['election tracker'])

  def name(self) -> str:
    return 'State of election'

  def state(self) -> str:
    return self._state

  def update(self) -> None:
    pass

  def get_vote_count(self) -> dict[str, int]:
    return self._vote_count

  def partial_state(
      self,
      player_name: str,
  ) -> str:
    """Return a player-specific view of the construct's state."""
    return self._state

  def update_after_event(
      self,
      event_statement: str,
  ) -> None:
    if not self._polls_open:
      update_log = {
          'date': self._clock_now(),
          'Summary': 'Polls are not open.',
          'Vote count': str(self._vote_count),
      }
      self._history.append(update_log)
      return

    chain_of_thought = interactive_document.InteractiveDocument(self._model)
    chain_of_thought.statement(event_statement)
    chain_of_thought.statement(f'List of citizens: {self._voter_names}')
    active_voter_id = chain_of_thought.multiple_choice_question(
        question='In the above transcript, which citizen took an action?',
        answers=self._voter_names,
    )
    vote = None
    active_voter = self._voter_names[active_voter_id]
    if active_voter not in self._citizens_who_already_voted:
      did_vote = chain_of_thought.yes_no_question(
          question=f'Did {active_voter} vote in the above transcript?'
      )
      if did_vote:
        question = (
            f'Current activity: {event_statement}.\nGiven the above, who whould'
            f' {active_voter} vote for?'
        )
        action_spec = agent_lib.ActionSpec(
            call_to_action=question,
            output_type='CHOICE',
            options=self._candidates,
            tag='vote',
        )
        vote = self._voter_by_name[active_voter].act(action_spec)
        action_spec.validate(vote)

        self._vote_count[vote] += 1
        self._citizens_who_already_voted.add(active_voter)
        self._memory.add(
            f'{active_voter} voted for {vote}', tags=['election tracker']
        )
        if self._verbose:
          print(
              termcolor.colored(
                  f'\n {active_voter} voted for {vote}\n', 'magenta'
              )
          )
      else:
        if self._verbose:
          print(
              termcolor.colored(
                  f'\n {active_voter} did not vote in the transcript.\n',
                  'magenta',
              )
          )
    else:
      chain_of_thought.statement(f'{active_voter} already voted.')

    update_log = {
        'date': self._clock_now(),
        'Summary': str(self._vote_count),
        'Vote count': str(self._vote_count),
        'Chain of thought': {
            'Summary': 'Election tracker chain of thought',
            'Chain': chain_of_thought.view().text().splitlines(),
        },
    }
    self._history.append(update_log)

    if self._verbose:
      print(
          termcolor.colored(
              f'{self._vote_count}\n' + chain_of_thought.view().text(),
              'magenta',
          )
      )

    if self._measurements is not None and vote is not None:
      answer = self._vote_count[vote]
      answer_str = str(answer)
      datum = {
          'time_str': self._clock_now().strftime('%H:%M:%S'),
          'timestep': self._timestep,
          'value_float': answer,
          'value_str': answer_str,
          'player': vote,
      }
      self._measurements.publish_datum(channel=self._channel, datum=datum)
      self._timestep += 1

In [13]:
# @title Create game master memory
game_master_memory = associative_memory.AssociativeMemory(
    embedder, importance_model_gm.importance, clock=clock.now)

In [ ]:
# @title Initialize game master memories
for player in players:
  game_master_memory.add(f'{player.name} is at their private home.')

In [15]:
# @title Create components and externalities
player_names = [player.name for player in players]

facts_on_village = components.constant.ConstantComponent(
    ' '.join(shared_memories), 'General knowledge of Riverbend')
player_status = gm_components.player_status.PlayerStatus(
    clock.now, model, game_master_memory, player_names)

relevant_events = gm_components.relevant_events.RelevantEvents(
    clock.now, model, game_master_memory)
time_display = gm_components.time_display.TimeDisplay(clock)

election_externality = Elections(
    model=model,
    clock_now=clock.now,
    memory=game_master_memory,
    voters=players,
    candidates=['Alice', 'Bob'],
    verbose=True,
    measurements=measurements,
)

mem_factory = blank_memories.MemoryFactory(
    model,
    embedder,
    importance_model_gm.importance,
    clock_now=clock.now,
)

convo_externality = gm_components.conversation.Conversation(
    players=players,
    model=model,
    memory=game_master_memory,
    clock=clock,
    burner_memory_factory=mem_factory,
    components=[player_status],
    cap_nonplayer_characters=2,
    shared_context=shared_context,
    verbose=True,
)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players,
    model=model,
    memory=game_master_memory,
    clock_now=clock.now,
    verbose=False,
    components=[player_status]
)

TIME_POLLS_OPEN = datetime.datetime(hour=14, year=2024, month=10, day=1)
TIME_POLLS_CLOSE = datetime.datetime(hour=20, year=2024, month=10, day=1)
schedule = {
    'start': gm_components.schedule.EventData(
        time=START_TIME,
        description='',
    ),
    'election': gm_components.schedule.EventData(
        time=datetime.datetime(hour=13, year=2024, month=10, day=1),
        description=(
            'The town of Riverbend is now holding an election to determine ' +
            'who will become the mayor. ' +
            f'Polls will open at {TIME_POLLS_OPEN}.'),
    ),
    'election_polls_open': gm_components.schedule.EventData(
        time=TIME_POLLS_OPEN,
        description=(
            'The election is happening now. Polls are open. Everyone may ' +
            'go to a polling place and cast their vote. ' +
            f'Polls will close at {TIME_POLLS_CLOSE}.'),
        trigger=election_externality.open_polls,
    ),
    'election_polls_close': gm_components.schedule.EventData(
        time=TIME_POLLS_CLOSE,
        description=(
            'The election is over. Polls are now closed. The results will ' +
            'now be tallied and a winner declared.'),
        trigger=election_externality.declare_winner,
    )
}

schedule_construct = gm_components.schedule.Schedule(
    clock_now=clock.now, schedule=schedule)


In [16]:
# @title Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    components=[
        facts_on_village,
        player_status,
        schedule_construct,
        election_externality,
        convo_externality,
        direct_effect_externality,
        relevant_events,
        time_display,
    ],
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
)

## The RUN

In [17]:
clock.set(START_TIME)

In [ ]:
# @title Expect about 2-3 minutes per step.

episode_length = 12  # @param {type: 'integer'}
for _ in range(episode_length):
  env.step()


In [ ]:
# @title Metrics plotting

group_by = collections.defaultdict(lambda: 'player')

# available_channels = list(measurements.available_channels())
available_channels = ['RationalityMetric', 'CommonSenseMorality']

fig, ax = plt.subplots(1, len(available_channels), figsize=(6, 2))
tb = [channel for channel in available_channels]
for idx, channel in enumerate(available_channels):
  plotting.plot_line_measurement_channel(measurements, channel,
                                         group_by=group_by[channel],
                                         xaxis='time_str',
                                         ax=ax[idx])
  ax[idx].set_title(channel)

fig.set_constrained_layout(constrained=True)

## Summary and analysis of the episode

In [ ]:
# @title Summarize the entire story.
all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

detailed_story = '\n'.join(all_gm_memories)
print('len(detailed_story): ', len(detailed_story))
# print(detailed_story)

episode_summary = model.sample_text(
    f'Sequence of events:\n{detailed_story}'+
    '\nNarratively summarize the above temporally ordered ' +
    'sequence of events. Write it as a news report. Summary:\n',
     max_tokens=3500, terminators=())
print(episode_summary)

In [ ]:
# @title Summarise the perspective of each player
player_logs = []
player_log_names = []
for player in players:
  name = player.name
  detailed_story = '\n'.join(memories[player.name].retrieve_recent(
      k=1000, add_time=True))
  summary = ''
  summary = model.sample_text(
      f'Sequence of events that happened to {name}:\n{detailed_story}'
      '\nWrite a short story that summarises these events.\n'
      ,
       max_tokens=3500, terminators=())

  all_player_mem = memories[player.name].retrieve_recent(k=1000, add_time=True)
  all_player_mem = ['Summary:', summary, 'Memories:'] + all_player_mem
  player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
  player_logs.append(player_html)
  player_log_names.append(f'{name}')


#Build and display HTML log of the experiment

In [ ]:
history_sources = [
    env,
    direct_effect_externality,
    convo_externality,
    election_externality,
]
histories_html = [
    html_lib.PythonObjectToHTMLConverter(history.get_history()).convert()
    for history in history_sources]
histories_names = [history.name for history in history_sources]

In [ ]:
gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

tabbed_html = html_lib.combine_html_pages(
    histories_html + [gm_mem_html] + player_logs,
    histories_names + ['GM'] + player_log_names,
    summary=episode_summary,
    title='Riverbend elections experiment',
)

tabbed_html = html_lib.finalise_html(tabbed_html)

In [ ]:
display.HTML(tabbed_html)

```
Copyright 2023 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```